In [1]:
import numpy as np
import math

In [2]:
cG = [0,0,0]
#x, y, z, Fxd, Fyd, Fzd 
locations = [[9,19.5,0, -1, 1, 0],
             [-9,19.5,0, 1, 1, 0],
             [-9,-19.5,0, -1, 1, 0],
             [9,-19.5,0, 1, 1, 0],
             [9,0,  0, 0, 0, 1],
             [-9,0,0, 0, 0, 1]]

theta = math.pi/4 #X-Y Plane
alpha = math.pi/2 #Vertical
coefficients = [[0 for i in range(0, len(locations))] for j in range(0,6)] 
sums = []

def findForce(motornumber): 
    forcez = locations[motornumber][5] * math.sin(alpha)
    rem = (1 - forcez ** 2) ** 0.5
    forcex = rem * locations[motornumber][3] * math.cos(theta)
    forcey = rem * locations[motornumber][4] * math.sin(theta) 
    return [forcex,forcey, forcez]

def findRadius(motornumber): 
    values = []
    for index in range(0, len(cG)): 
        values.append(locations[motornumber][index] - cG[index])
    return values

def findTorque(radius, force): 
    torx = radius[1] * force[2] - radius[2] * force[1]
    tory = radius[2] * force[0] - radius[0] * force[2]
    torz = radius[0] * force[1] - radius[1] * force[0]
    return torx, tory, torz

def set_column(column):
    height = 0
    forcev = findForce(column)
    for item in forcev: 
        coefficients[height][column] = item
        height += 1
    torques = findTorque(findRadius(column), forcev)
    for item in torques: 
        coefficients[height][column] = item
        height += 1

def disablemotor(motornumber):
    off = True
    for index in range(0,3):
        if(coefficients[index][motornumber] != 0): 
            off = False
            break
    if(off): 
        set_column(motornumber)
    else:
        for index in range(0, 6):
            coefficients[index][motornumber] = 0

def update_coefficients():
    for column in range(len(locations)): 
        set_column(column)
    for i in range(0, len(coefficients)): 
        value = 0
        for j in coefficients[i]: 
            value += abs(j)
        sums.append(value)

def print_coefficient(n): 
    for row in coefficients:
        t = ""
        for value in row:
            t += str(round(value,n)) + " "
        print(t)

update_coefficients()

In [3]:
maxspeed = 250
threshold = 0.05    

def solve_system(solutions): 
    while(len(solutions) < len(coefficients)): 
        solutions.append(0)
    solutions = [solutions[i] * sums[i] for i in range(len(sums))]
    thrusts = np.linalg.lstsq(coefficients, solutions, rcond= None)[0]
    formatted = []
    for index in range(len(thrusts)): 
        if(abs(thrusts[index]) <= threshold): 
            formatted.append(0)
        else: 
            value = max(min(thrusts[index], 1),-1)
            formatted.append(int(round(maxspeed * value)))
    return formatted

disablemotor(0)
print_coefficient(3)
print()
for index in range(0,6): 
    x = [0 for i in range(0,6)]
    x[index] = 1
    print(solve_system(x))

0 0.707 -0.707 0.707 0.0 0.0 
0 0.707 0.707 0.707 0.0 0.0 
0 0.0 0.0 0.0 1.0 1.0 
0 0.0 -0.0 -0.0 0.0 0.0 
0 0.0 0.0 0.0 -9.0 9.0 
0 -20.153 -20.153 20.153 0.0 -0.0 

[0, 250, -250, 0, 0, 0]
[0, 0, 250, 250, 0, 0]
[0, 0, 0, 0, 250, 250]
[0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, -250, 250]
[0, -250, 0, 250, 0, 0]
